# Summary

In this notebook we learn both single-scale and multiscale DAGs, by using MSCASTLE. 

In [1]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.insert(0, r"..\code")

from MSCASTLE import SSCASTLE, MSCASTLE
from utils import save_obj,load_obj

In [2]:
data_dir="../data/"
ts_dir="../data/TimeSeriesDiffRegion/" 
save_dir="../data/processed/diff_regions/"

In [3]:
replace = False #set True to overwrite existing results
verbose = False #set True to print optimization info
transform='swt' #wavelet transform
wv='db5' #wavelet family
ndetails=5 #number of details of the wavelet transform

In [4]:
counter=0

for filename in os.listdir(ts_dir):
    counter+=1
    print("\n\n+++++++++++++++++++")
    print("Doing file n{}".format(counter))
    print("+++++++++++++++++++")
    
    f = os.path.join(ts_dir, filename)
    
    if os.path.isfile(f):
    
        print("\n\nStarting {}".format(filename))
        nn=filename.split('.pkl')[0]
        subj=nn.split('_')[1]
        test=nn.split('_')[3]
        side=nn.split('_')[-1]

        name_save_SS = 'res_singlescale_Subj_{}_exam{}_side{}'.format(subj,test,side)
        name_save_MS = 'res_multiscale_Subj_{}_exam{}_side{}_transform{}_wv{}_ndetails{}'.format(subj,test,side,transform,wv,ndetails)
        
        #load data
        df=load_obj(nn, data_dir=ts_dir)
        
        #single scale
        print("++++++++++++++++++ SINGLE SCALE ++++++++++++++++++")
        try:
            if not replace:
                if os.path.exists(os.path.join(save_dir, name_save_SS+'.pkl')):
                    print("SSCASTLE results for subject {} (exam{}, side{}) are already available. Skipping to MSCASTLE.".format(subj,test,side))
                else:
                    sscastle = SSCASTLE(df.values)
                    
                    if verbose:
                        Cs, (it, t), loss, ell1 = sscastle.solver(reg='l1', lmbd=.1, thresh=.1, interval=(-np.inf, np.inf), verbose=verbose)
                        res = {'causal_interactions':Cs, 'Opt it/time': (it, t), 'Final loss': loss, 'ell1 norm':ell1}
                    else:
                        Cs, (it, t)= sscastle.solver(reg='l1', lmbd=.1, thresh=.1, interval=(-np.inf, np.inf), verbose=verbose)
                        res = {'causal_interactions':Cs, 'Opt it/time': (it, t)}
                    
                    save_obj(res, name_save_SS, data_dir=save_dir)
                    print("SSCASTLE results correctly saved.")
            else:
                sscastle = SSCASTLE(df.values)
                
                if verbose:
                    Cs, (it, t), loss, ell1 = sscastle.solver(reg='l1', lmbd=.1, thresh=.1, interval=(-np.inf, np.inf), verbose=verbose)
                    res = {'causal_interactions':Cs, 'Opt it/time': (it, t), 'Final loss': loss, 'ell1 norm':ell1}
                else:
                    Cs, (it, t)= sscastle.solver(reg='l1', lmbd=.1, thresh=.1, interval=(-np.inf, np.inf), verbose=verbose)
                    res = {'causal_interactions':Cs, 'Opt it/time': (it, t)}
                
                save_obj(res, name_save_SS, data_dir=save_dir)
                print("SSCASTLE results correctly saved.")
        except Exception as e:
            print("SSCASTLE failed for subject {}, exam {}, side {}".format(subj, test,side))
            print(e)
            pass

        #multi scale
        print("++++++++++++++++++ MULTI SCALE ++++++++++++++++++")
        try:
            if not replace:
                if os.path.exists(os.path.join(save_dir, name_save_MS+'.pkl')):
                    print("MSCASTLE results for subject {} (exam{}, side{}, wv{}) are already available. Skipping to next dataset.".format(subj,test, side, wv))
                else:
                    cut_df = df.iloc[8:-8].copy()
                    T,N = cut_df.shape
                    mscastle = MSCASTLE(cut_df.values, multiscale=True, transform=transform, wavelet=wv, lag=0, ndetails=ndetails)

                    Cs_ms = mscastle._solver(reg='l1', lmbd=.1, thresh=.1, interval=(-np.inf,np.inf), verbose=verbose)
                    res_ms = {'causal_interactions':Cs_ms}
                    save_obj(res_ms, name_save_MS, data_dir=save_dir)
                    print("MSCASTLE results correctly saved.")
            else:
                cut_df = df.iloc[8:-8].copy()
                T,N = cut_df.shape
                mscastle = MSCASTLE(cut_df.values, multiscale=True, transform=transform, wavelet=wv, lag=0, ndetails=ndetails)
                
                Cs_ms = mscastle._solver(reg='l1', lmbd=.1, thresh=.1, interval=(-np.inf,np.inf), verbose=verbose)
                res_ms = {'causal_interactions':Cs_ms}
                save_obj(res_ms, name_save_MS, data_dir=save_dir)
                print("MSCASTLE results correctly saved.")

        except Exception as e:
            print("MSCASTLE failed for subject {}, exam {}, side {}, wv {}".format(subj, test, side, wv))
            print(e)
            pass



+++++++++++++++++++
Doing file n1
+++++++++++++++++++


Starting Subj_100307_AAL89_Exam0_TS_HCP_L.pkl
++++++++++++++++++ SINGLE SCALE ++++++++++++++++++
SSCASTLE results for subject 100307 (examExam0, sideL) are already available. Skipping to MSCASTLE.
++++++++++++++++++ MULTI SCALE ++++++++++++++++++
MSCASTLE results for subject 100307 (examExam0, sideL, wvdb5) are already available. Skipping to next dataset.


+++++++++++++++++++
Doing file n2
+++++++++++++++++++


Starting Subj_100307_AAL89_Exam0_TS_HCP_R.pkl
++++++++++++++++++ SINGLE SCALE ++++++++++++++++++
SSCASTLE results for subject 100307 (examExam0, sideR) are already available. Skipping to MSCASTLE.
++++++++++++++++++ MULTI SCALE ++++++++++++++++++
MSCASTLE results for subject 100307 (examExam0, sideR, wvdb5) are already available. Skipping to next dataset.


+++++++++++++++++++
Doing file n3
+++++++++++++++++++


Starting Subj_100307_AAL89_Exam1_TS_HCP_L.pkl
++++++++++++++++++ SINGLE SCALE ++++++++++++++++++
SSCASTLE resu